# Function Calling

# Overview
Function calling in OpenAI allows you to integrate AI-generated text with external functions. \
This enables you to execute code, perform calculations, or fetch data based on the model's output. \
By defining functions and allowing the model to call them, you can create more interactive and dynamic applications.

For more detailed examples, refer to the official documentation [Function Calling with Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython)



In [29]:
import os
from openai import AzureOpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)
CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_NAME')

## Define functions that will take actions like retrieve information from internet

In [30]:


# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})



## Let's ask a question

In [114]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role":"system", "content":"""You are an AI assistant designed to help users get information about the weather. 
                 When a user asks for information about the weather, you should use the tool function named 'get_current_weather'. 
                 Only use the functions you have been provided with.
                Don't make assumptions about what values to plug into functions. 
                 Ask for clarification if a user request is ambiguous.
                 The format of the arguments should not contain '/*' and be a valid JSON string with property names enclosed in double quotes."""},
                 {"role": "user", "content": "What's the weather like in San Francisco?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=messages,
        tools=tools,
        tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # Sometimes the name is classified mistakenly as python
            if function_name == "python":
                function_name = "get_current_weather"
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VUwpgt7KLVLcFrkUV394ojmS', function=Function(arguments='{\n    "location": "San Francisco",\n    "unit": "fahrenheit"\n}', name='get_current_weather'), type='function')])
[ChatCompletionMessageToolCall(id='call_VUwpgt7KLVLcFrkUV394ojmS', function=Function(arguments='{\n    "location": "San Francisco",\n    "unit": "fahrenheit"\n}', name='get_current_weather'), type='function')]
{
    "location": "San Francisco",
    "unit": "fahrenheit"
}
ChatCompletion(id='chatcmpl-9f0IFQCXDTsvAzeVtxyCRXfwwiZSp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in San Francisco is 72 degrees Fahrenheit.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtere

#### Use cases to explore and think about with the customer
1. **Dynamic Calculations**  \
Perform real-time calculations based on user input.

2. **Data Fetching** \
Retrieve data from external APIs based on the model's output.

3. **Interactive Applications** \
Create more interactive and dynamic applications by integrating AI with backend functions.

#### Best Practices
1. **Clear Prompts** \
Ensure prompts are clear and specific to get accurate responses from the model.

2. **Robust Parsing** \
Implement robust parsing logic to handle various formats of model output.

3. **Error Handling** \
Include error handling to manage unexpected outputs or function call failures.